In [1]:
import numpy as np
import pandas as pd
import json
from pandas.io.json import json_normalize 
from pandas import read_csv
import logging
from functools import reduce
import csv

/Users/ghost/src/virtual_env/jupyther3.6/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/ghost/src/virtual_env/jupyther3.6/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
## Configuration 

In [3]:
#etl = 'kobo2elastic'
#etl = 'curis2elastic'
#etl = 'oldcuris2newcuris'
etl = 'isabela2newaqm'
#etl = 'cambodia2newaqm'

input_schema_file = ''
input_data_file = ''
mapping_file = ''

root_object = 'resident'

if etl == 'curis2elastic':
    #old curis to elasticsearch
    input_schema_file = 'schema/input/curisSchema.1-item.json'
    input_data_file = 'data/curisData.1-items.json'
    mapping_file = 'schema/map/couchbase2elastic.map.csv'
    output_dir = 'file/curisSchema/'
    
elif etl == 'kobo2elastic':
    #kobo to elasticsearch
    input_schema_file = 'schema/input/koboSchema.1-item.json'
    input_data_file = 'data/koboData.2-items.json'
    mapping_file = 'schema/map/kobo2elastic.map.csv'
    output_dir = 'file/koboSchema/'
    
elif etl == 'oldcuris2newcuris':
    #kobo to elasticsearch
    input_schema_file = 'schema/input/curisSchema.1-item.json'
    input_data_file = 'data/curisData.1-item.json'
    mapping_file = 'schema/map/kobo2elastic.map.csv'
    output_dir = 'file/curisSchema/'
    
elif etl == 'cambodia2newaqm':
    #kobo to elasticsearch
    input_schema_file = 'schema/input/KHMSchema.json'
    input_data_file = 'data/KHM.3-items.json'
    mapping_file = 'schema/map/kobo2elastic.map.csv'
    output_dir = 'file/KHMSchema/'

elif etl == 'isabela2newaqm':
    #kobo to elasticsearch
    input_schema_file = 'schema/input/curisSchema.1-item.json'
    input_data_file = 'data/Isabela.2-items.json'
    mapping_file = 'schema/map/Isabela2newAQM.map.csv'
    #output_dir = 'file/IsabelaSchema/'
    output_dir = 'data/processed/couchbase-curis-2019-06-21/'

In [4]:
## Flatten json

In [5]:
def _flatten_json(nested_json):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '.')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + '' + str(i) + '.')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(nested_json)
    return out

In [6]:
## header filters

In [7]:
## OPTIMIZE TO ACCEPT LIST NOT DATAFRAME
def filter_key(x):
    lists = format_key(x)
    lists = exclude_digit(lists)
    lists = list2string(lists)
    return lists

def filter_index_map(x):
    lists = format_key(x)
    lists = include_digit(lists)
    lists = list2string(lists)
    return lists

def format_key(items):
    #return list(map(lambda x:x.lower().split(sep='.'), items ))
    return items.lower().split(sep='.')

def include_digit(items):
    return [item for item in items if item.isdigit()]

def exclude_digit(items):
    return [item for item in items if not item.isdigit()]

def list2string(lists):
    return '.'.join(lists)

In [8]:
## File loader

In [9]:
def _open_file():
    with open(input_data_file) as f:
        return json.load(f)
    
def _dict_to_dataframe(dict_object):
    return pd.DataFrame.from_dict({'value': dict_object})

def _get_mapping_fields():
    _mapping_df = pd.DataFrame()
    _mapping_df = read_csv(mapping_file).sort_values(['source_key'])

    _required_mapping_fields = list(_mapping_df['source_key'])
    return _required_mapping_fields

def _add_custom_colums(dataframe_object):
    dataframe_object['key'] = list(dataframe_object.index)
    dataframe_object['key'] = dataframe_object['key'].apply(filter_key)
    #print('_add_custom_columns ',dataframe_object[dataframe_object['key'] == 'id' ])

    dataframe_object['_index_map'] = list(dataframe_object.index)
    dataframe_object['_index_map'] = dataframe_object['_index_map'].apply(filter_index_map)
    #TODO: musbe adjusted dynamically
    
    #CHECK if single object or array of objects
    #formid == koboschema
    #id == curischema
    #dataframe_object['_id'] = dataframe_object[dataframe_object['key'] == 'formid' ]['value'].values[0]
    
    #IF oldcursi2newcuris
    dataframe_object['_id'] = dataframe_object[dataframe_object['key'] == 'id' ]['value'].values[0]
    #IF KHMSChema
    #dataframe_object['_id'] = dataframe_object[dataframe_object['key'] == '_id' ]['value'].values[0]
    
    #print(dataframe_object)
    dataframe_object = dataframe_object.reset_index(drop=True)

    return dataframe_object

In [10]:
def _get_csv_headers(filename):
    return pd.read_csv( output_dir + filename + '.csv',nrows=0) #get header only

In [11]:
def _get_required_data(input_data_df,csv_header_df):
    
    required_header_list = [] 

    for data in input_data_df['key']:
        
        if data in csv_header_df.columns:
            
            required_header_list.append(data)
    
    #print(required_header_list)
    
    return input_data_df[input_data_df['key'].isin(required_header_list)]

def _rename_index(data_df):
    if 'key' in data_df.columns:
        data_df.index = list(data_df['key'])
        data_df = data_df.drop('key', axis=1)
    return data_df 

def _columnar_to_row(input_data_df,csv_header_df):
    
    for header in list(input_data_df.index):
        if header:
            csv_header_df.at['',header] = input_data_df.loc[header]['value'] 
    

        csv_header_df.at['','_id'] = input_data_df['_id'][header] 
        csv_header_df.at['','_index_map'] = input_data_df['_index_map'][header] 
    return csv_header_df
    
def _write_to_csv(data_csv_df, filename):
    return data_csv_df.to_csv(output_dir + filename + '.csv', encoding='utf-8', mode='a', header=False,index=False)

In [44]:
def _get_index_map(data_df):
    return list(data_df['_index_map'].unique())

def _get_filenames():
    schema_csv = pd.read_csv(output_dir + 'schema.csv', skiprows=0)
    return schema_csv['file_name'].values[0].split(sep=",")

def _filter_by_indexmap(data_df, index):
    return data_df[data_df['_index_map'] == index ]

In [76]:
def clean_value(x):
    return x.lower().replace("/", ".").replace("the", "").replace("schema", "").strip()

In [75]:
def _main(data_flat_df):
    index_map_list = []
    headers_list = []
    
    index_map_list = _get_index_map(data_flat_df)
    headers_list = _get_filenames()
    
    
    _required_mapping_fields = []
    _required_mapping_fields = _get_mapping_fields()

    print('index_map_list :',index_map_list)
    
    ###SHIT ERROR ITERATION HERE
    
    for index in index_map_list:
        for header in headers_list:
            
            #print('index :', index)
            source_data_df = pd.DataFrame()
            source_data_df = _filter_by_indexmap(data_flat_df, index)
            
            #source_data_df['key']  =  source_data_df['key'].apply(clean_value)
            source_data_df['key'].apply(clean_value)
            
            csv_header_df = pd.DataFrame()
            csv_header_df = _get_csv_headers(header)
            
            new_data_df = pd.DataFrame()
            #print('index')
            '''
            new_data_df = _get_required_data(source_data_df, csv_header_df)
            
            new_data_df = _rename_index(new_data_df)
            new_data_df = _columnar_to_row(new_data_df, csv_header_df)
            new_data_df = _write_to_csv(new_data_df, header)
            
            '''
    
    return new_data_df
    
    

In [15]:
def init(data_list):
    index = 0
    print('total item: ', len(data_list))
    
    ##FILTER HERE ONLY THE REQUIRED FIELDS
    _required_mapping_fields = []
    _required_mapping_fields = _get_mapping_fields()

    for datum in data_list:
        index += 1
        

        data_flat_dict = {}
        data_flat_df = pd.DataFrame()
        
        data_flat_dict = _flatten_json(datum)
        #print(data_flat_dict)
        
        data_flat_df = _dict_to_dataframe(data_flat_dict)
        
        
        data_flat_df = _add_custom_colums(data_flat_df)
        
        ##FILTER HERE ONLY THE REQUIRED FIELDS
        #_required_mapping_fields = _get_mapping_fields()
        #data_flat_df = data_flat_df[data_flat_df.index.isin(_required_mapping_fields)]
        
        data_flat_df[data_flat_df['key'].isin(_required_mapping_fields)]
        
        _main(data_flat_df)
    return data_flat_df

In [16]:
data_list = []
data_list = _open_file()
#data_list

In [72]:
data_flat_df = init(data_list)
data_flat_df
#data_list

total item:  2
index_map_list : ['0', '1', '2', '3', '4', '5', '', '0.0', '0.1', '0.2']
clean_vlaue:  _sync.history.channels
clean_vlaue:  _sync.history.parents
clean_vlaue:  _sync.history.revs
clean_vlaue:  _sync.recent_sequences
clean_vlaue:  households.date_updated
clean_vlaue:  households.house_ownership
clean_vlaue:  households.neighborhood_description
clean_vlaue:  households.no_of_families_in_the_household
clean_vlaue:  households.no_of_people_in_the_household
clean_vlaue:  households.sanitary_ownership
clean_vlaue:  households.type_of_house
clean_vlaue:  profiles.civil_status
clean_vlaue:  profiles.date_updated
clean_vlaue:  profiles.education
clean_vlaue:  profiles.employment.is_employed
clean_vlaue:  profiles.employment.monthly_income
clean_vlaue:  profiles.employment.nature
clean_vlaue:  profiles.religion
index
clean_vlaue:  _sync.history.channels
clean_vlaue:  _sync.history.parents
clean_vlaue:  _sync.history.revs
clean_vlaue:  _sync.recent_sequences
clean_vlaue:  household

,value,key,_index_map,_id
0,None,_sync.history.channels,0,2f920d39-319e-4bf7-90c8-6133c3337af7
1,None,_sync.history.channels,1,2f920d39-319e-4bf7-90c8-6133c3337af7
2,None,_sync.history.channels,10,2f920d39-319e-4bf7-90c8-6133c3337af7
3,None,_sync.history.channels,11,2f920d39-319e-4bf7-90c8-6133c3337af7
4,None,_sync.history.channels,12,2f920d39-319e-4bf7-90c8-6133c3337af7
5,None,_sync.history.channels,13,2f920d39-319e-4bf7-90c8-6133c3337af7
6,None,_sync.history.channels,2,2f920d39-319e-4bf7-90c8-6133c3337af7
7,None,_sync.history.channels,3,2f920d39-319e-4bf7-90c8-6133c3337af7
8,None,_sync.history.channels,4,2f920d39-319e-4bf7-90c8-6133c3337af7
9,None,_sync.history.channels,5,2f920d39-319e-4bf7-90c8-6133c3337af7


## DICT TO DATAFRAME

In [28]:
datum = data_list[0]
test_data_flat_dict = _flatten_json(datum)    
test_data_flat_df = _dict_to_dataframe(test_data_flat_dict)
test_data_flat_df

,value
_sync.history.channels.0,None
_sync.history.channels.1,None
_sync.history.channels.2,None
_sync.history.channels.3,None
_sync.history.channels.4,None
_sync.history.channels.5,None
_sync.history.parents.0,5
_sync.history.parents.1,-1
_sync.history.parents.2,1
_sync.history.parents.3,2


## ADD CUSTOM COLUMNS WITHOUT MAPPING

In [29]:
_customed_data_flat_df = _add_custom_colums(test_data_flat_df)
_customed_data_flat_df

,value,key,_index_map,_id
0,None,_sync.history.channels,0,64c2e7de-0cc8-4df5-a54f-02398652c080
1,None,_sync.history.channels,1,64c2e7de-0cc8-4df5-a54f-02398652c080
2,None,_sync.history.channels,2,64c2e7de-0cc8-4df5-a54f-02398652c080
3,None,_sync.history.channels,3,64c2e7de-0cc8-4df5-a54f-02398652c080
4,None,_sync.history.channels,4,64c2e7de-0cc8-4df5-a54f-02398652c080
5,None,_sync.history.channels,5,64c2e7de-0cc8-4df5-a54f-02398652c080
6,5,_sync.history.parents,0,64c2e7de-0cc8-4df5-a54f-02398652c080
7,-1,_sync.history.parents,1,64c2e7de-0cc8-4df5-a54f-02398652c080
8,1,_sync.history.parents,2,64c2e7de-0cc8-4df5-a54f-02398652c080
9,2,_sync.history.parents,3,64c2e7de-0cc8-4df5-a54f-02398652c080


## REQUIRED MAPPING

In [31]:
_required_mapping_fields = []
_required_mapping_fields = _get_mapping_fields()
_required_mapping_fields

['address.barangay',
 'address.country',
 'address.lot_or_house_number',
 'address.postal_code',
 'address.province',
 'birthdate',
 'contact_number.number',
 'email_address',
 'family_members',
 'first_name',
 'gender',
 'households.house_ownership',
 'households.neighborhood_description',
 'households.sanitary_ownership',
 'id',
 'last_name',
 'middle_name',
 'profiles.civil_status',
 'registered_at',
 'user-cam.id',
 nan]

In [42]:
_customed_data_flat_df = _customed_data_flat_df[_customed_data_flat_df['key'].isin(_required_mapping_fields)]
_customed_data_flat_df

,value,key,_index_map,_id
22,Bangad,address.barangay,,64c2e7de-0cc8-4df5-a54f-02398652c080
23,Philippines,address.country,,64c2e7de-0cc8-4df5-a54f-02398652c080
24,,address.lot_or_house_number,,64c2e7de-0cc8-4df5-a54f-02398652c080
25,3330,address.postal_code,,64c2e7de-0cc8-4df5-a54f-02398652c080
26,Isabela,address.province,,64c2e7de-0cc8-4df5-a54f-02398652c080
27,08/09/1989,birthdate,,64c2e7de-0cc8-4df5-a54f-02398652c080
29,None,contact_number.number,,64c2e7de-0cc8-4df5-a54f-02398652c080
30,None,email_address,,64c2e7de-0cc8-4df5-a54f-02398652c080
31,,first_name,,64c2e7de-0cc8-4df5-a54f-02398652c080
32,Female,gender,,64c2e7de-0cc8-4df5-a54f-02398652c080


## FILTER BY INDEX_MAP

In [97]:
test_source_data_df = pd.DataFrame()
test_source_data_df = _filter_by_indexmap(_customed_data_flat_df, '0')
test_source_data_df

,value,key,_index_map,_id
37,Owned,households.house_ownership,0,64c2e7de-0cc8-4df5-a54f-02398652c080
38,Rural,households.neighborhood_description,0,64c2e7de-0cc8-4df5-a54f-02398652c080
41,Owned,households.sanitary_ownership,0,64c2e7de-0cc8-4df5-a54f-02398652c080
58,Single,profiles.civil_status,0,64c2e7de-0cc8-4df5-a54f-02398652c080


## CLEAN FILTERED BY INDEX_MAP

In [98]:
test_source_data_df['key'].apply(clean_value)
test_source_data_df

,value,key,_index_map,_id
37,Owned,households.house_ownership,0,64c2e7de-0cc8-4df5-a54f-02398652c080
38,Rural,households.neighborhood_description,0,64c2e7de-0cc8-4df5-a54f-02398652c080
41,Owned,households.sanitary_ownership,0,64c2e7de-0cc8-4df5-a54f-02398652c080
58,Single,profiles.civil_status,0,64c2e7de-0cc8-4df5-a54f-02398652c080


## GET HEADER_LIST CSV

In [79]:
test_headers_list = _get_filenames()
test_headers_list

['date_visits',
 'family_members',
 'health_informations',
 'health_informations.diagnosed',
 'health_informations.family_history',
 'households',
 'households.amenities_present_in_house',
 'households.sanitary_type',
 'profiles',
 'symptoms_collection',
 'symptoms_collection.abdomen.abdomen',
 'symptoms_collection.arms.elbow',
 'symptoms_collection.arms.fingers',
 'symptoms_collection.arms.hand_and_palm',
 'symptoms_collection.arms.lower_arm',
 'symptoms_collection.arms.upper_arm',
 'symptoms_collection.arms.wrist',
 'symptoms_collection.chest.chest',
 'symptoms_collection.chest.lungs_and_breathing',
 'symptoms_collection.head.chin_and_jaw',
 'symptoms_collection.head.ears',
 'symptoms_collection.head.eyes',
 'symptoms_collection.head.head',
 'symptoms_collection.head.mouth',
 'symptoms_collection.head.nose',
 'symptoms_collection.legs.foot',
 'symptoms_collection.legs.knee',
 'symptoms_collection.legs.shin',
 'symptoms_collection.legs.thigh',
 'symptoms_collection.legs.toes',
 'sympt

## GET CSV_HEADER

In [101]:
test_csv_header_df = pd.DataFrame()
test_csv_header_df = _get_csv_headers('households')
test_csv_header_df

,_id,_index_map,households.type_of_house,households.house_ownership,households.sanitary_ownership,households.neighborhood_description,households.date_updated


## GET REQUIRED DATA

In [107]:
test_new_data_df = pd.DataFrame()
test_new_data_df = _get_required_data(test_source_data_df, test_csv_header_df)
test_new_data_df

,value,key,_index_map,_id
37,Owned,households.house_ownership,0,64c2e7de-0cc8-4df5-a54f-02398652c080
38,Rural,households.neighborhood_description,0,64c2e7de-0cc8-4df5-a54f-02398652c080
41,Owned,households.sanitary_ownership,0,64c2e7de-0cc8-4df5-a54f-02398652c080


## RENAME REQUIRED DATA

In [108]:
test_new_data_df = _rename_index(test_new_data_df)
test_new_data_df

,value,_index_map,_id
households.house_ownership,Owned,0,64c2e7de-0cc8-4df5-a54f-02398652c080
households.neighborhood_description,Rural,0,64c2e7de-0cc8-4df5-a54f-02398652c080
households.sanitary_ownership,Owned,0,64c2e7de-0cc8-4df5-a54f-02398652c080


## COLUMNAR TO ROW REQIURED DATA

In [109]:
test_new_data_df = _columnar_to_row(test_new_data_df, test_csv_header_df)
test_new_data_df

,_id,_index_map,households.type_of_house,households.house_ownership,households.sanitary_ownership,households.neighborhood_description,households.date_updated
,64c2e7de-0cc8-4df5-a54f-02398652c080,0,NaN,Owned,Owned,Rural,NaN


## MAPPING FIELDS

In [27]:
## =================================================

In [28]:
## Get Index_map unique values

In [29]:
## BUG: IF ADDRESS IS 
'''
    "answers": {
        "Address": [{
            "Location": {
                "Street": ["12312", "234234"],
                "City": "City"
            }
        }],
'''

'\n    "answers": {\n        "Address": [{\n            "Location": {\n                "Street": ["12312", "234234"],\n                "City": "City"\n            }\n        }],\n'